### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [1]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from demo.scenarios.session import *

Creating initial custom lists at URI: local:///Users/rbrowersinning/Documents/ResearchFolders/Continuum_LTP/GitRepos/mlte/demo/scenarios/../store
Loaded 7 qa_categories for initial list
Loaded 30 quality_attributes for initial list
Creating sample catalog at URI: StoreType.LOCAL_FILESYSTEM:local:///Users/rbrowersinning/Documents/ResearchFolders/Continuum_LTP/GitRepos/mlte/demo/scenarios/../store
Loading sample catalog entries.
Loaded 9 entries for sample catalog.


### Helper Functions
General functions and external imports.

In [2]:
# General functions.

from demo.scenarios import garden
import numpy as np
import pandas as pd
from scipy import stats
from os import path

def load_data(data_folder: str):
    """Loads all garden data results and taxonomy categories."""
    df_results = garden.load_base_results(data_folder, "predictions_test.csv")
    df_results.head()

    # Load the taxonomic data and merge with results.
    df_info = garden.load_taxonomy(data_folder)
    df_results.rename(columns={"label": "Label"}, inplace=True)
    df_all = garden.merge_taxonomy_with_results(df_results, df_info)

    return df_info, df_all

def load_results(data_folder: str):
    """loads reproducabilty test result runs"""
    #my_folder = data_folder + 
    df_results = pd.read_csv(path.join(data_folder, 'ReproducibilityDataSet_CV.csv'))

    return df_results
    #df_results = garden.load_base_results(data_folder,)


In [3]:
# Prepare the data. For this section, instead of executing the model, we will use CSV files containing the results of an already executed run of the model.
#df_info, df_all = load_data(DATASETS_DIR)

#df = pd.read_csv('ReproducibilityDataSet_CV.csv')

df = load_results(DATASETS_DIR)


In [4]:
results = stats.friedmanchisquare(df.Accuracy_r0, df.Accuracy_r1, df.Accuracy_r2, df.Accuracy_r3, df.Accuracy_r4, df.Accuracy_r5, df.Accuracy_r6, 
              df.Accuracy_r7, df.Accuracy_r8, df.Accuracy_r9)
results.pvalue

np.float64(0.9980720828075595)

### Measurements

In this first example, we simply wrap the output from `accuracy_score` with a custom `Result` type to cope with the output of a third-party library that is not supported by a MLTE builtin.

In [5]:
from mlte.evidence.types.array import Array
from mlte.measurement.external_measurement import ExternalMeasurement


kruskal_measurement = ExternalMeasurement(
    "repeated training on training samples", Array, stats.friedmanchisquare
)

# Evaluate.
kruskal_res = kruskal_measurement.evaluate(
    df.Accuracy_r0, df.Accuracy_r1, df.Accuracy_r2, df.Accuracy_r3, df.Accuracy_r4, df.Accuracy_r5, df.Accuracy_r6, 
    df.Accuracy_r7, df.Accuracy_r8, df.Accuracy_r9
)

# Inspect values
print(kruskal_res)

# Save to artifact store
kruskal_res.save(force=True)

FriedmanchisquareResult(statistic=np.float64(1.3575418994429564), pvalue=np.float64(0.9980720828075595))


ArtifactModel(header=ArtifactHeaderModel(identifier='evidence.repeated training on training samples', type='evidence', timestamp=1759166196, creator=None, level='version'), body=EvidenceModel(artifact_type=<ArtifactType.EVIDENCE: 'evidence'>, metadata=EvidenceMetadata(test_case_id='repeated training on training samples', measurement=MeasurementMetadata(measurement_class='mlte.measurement.external_measurement.ExternalMeasurement', output_class='mlte.evidence.types.array.Array', additional_data={'function': 'scipy.stats._stats_py.friedmanchisquare'})), evidence_class='mlte.evidence.types.array.Array', value=ArrayValueModel(evidence_type=<EvidenceType.ARRAY: 'array'>, data=[np.float64(1.3575418994429564), np.float64(0.9980720828075595)])))